In [1]:
import ee
# ee.Authenticate()
import geemap
import geemap.chart as chart
import os
geemap.ee_initialize()
# geemap.update_package()
# !pip install geemap rasterio fiona
import json
import pandas as pd
# !pip install seaborn
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import ipyleaflet
import ipywidgets as widgets
import math

In [2]:
# Define a Point object
poi = ee.Geometry.Point([48.116927, 31.272270]).buffer(2000)
GF = ee.Geometry.Point([48.116927, 31.272270]).buffer(50)

# Define Map and enable drawn features exports
Map = geemap.Map(basemap="HYBRID")
Map.centerObject(poi, 13)

# Create start and end dates
start = '2021-01-01
end = '2021-12-31'

In [3]:
# Define the visualization parameters

vizParamsS2 = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
};


vizParamsCO = {
    'min': 0,
    'max': 0.05,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}


vizParamsNO2 = {
    'min': 0,
    'max': 0.0003,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}


vizParamsUVAI = {
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'min': -1,
    'max': 1,
}


vizParamsCH4 = {
    'min': 1750,
    'max': 1900,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

In [4]:
# Load and filter Sentinel2 SR (5 days, 10-30m resolution) Image Collection

# 1) Create cloud mask
def maskS2clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    
    # Set to zero for clear conditions
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)


# 2) Load and filter image collection
S2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate(start, end) \
    .filterBounds(poi) \
    .map(maskS2clouds) \
#     .map(lambda image: image.clip(poi))


# 3) Load single date image
S2_img = ee.Image('COPERNICUS/S2_SR/20210405T072611_20210405T073144_T38RQV').select('B4', 'B3', 'B2').multiply(0.0001)


## Retrieve image (collection) property info
# S2.aggregate_array('system:index').getInfo()
# S2_img.propertyNames().getInfo()

In [7]:
# Load and filter TROPOMI Image Collections

# GEE CO: CO mol/m^2 total columns with sensitivity to the tropospheric boundary layer. 1113.2 meters.
collCO = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO') \
    .select('CO_column_number_density') \
    .filterDate(start, end) \
    .filterBounds(poi) \
#     .map(lambda image: image.clip(poi))


# GEE NO2: mol/m^2 concentrations of collective nitrogen oxides (sunlight converts NO into NO2 and vice versa on 
# a timescale of minutes. 1113.2 meters.
collNO2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2') \
    .select('tropospheric_NO2_column_number_density') \
    .filterDate(start, end) \
    .filterBounds(poi) \
#     .map(lambda image: image.clip(poi))


# GEE UVAI: Daily, calculated with a pair of measurements at the 354 nm and 388 nm, 1113.2 meters
collUVAI = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_AER_AI') \
    .select('absorbing_aerosol_index') \
    .filterDate(start, end) \
    .filterBounds(poi) \
#     .map(lambda image: image.clip(poi))
    

# GEE CH4: CH4 column concentrations in ppbV with high sensitivity to the Earth’s surface. 1113.2 meters.
collCH4 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CH4') \
    .select('CH4_column_volume_mixing_ratio_dry_air_bias_corrected') \
    .filterDate(start, end) \
    .filterBounds(poi) \
#     .map(lambda image: image.clip(poi))

In [8]:
# # Prepare time series for TROPOMI products

# # Extract image values in collection (function takes single image as input and reduces data to region)
# def CO_max(img):
#     max = img.reduceRegion(reducer=ee.Reducer.max(), geometry=poi, scale=1120).get('CO_column_number_density')
#     return img.set('date', img.date().format()).set('max',max)

# def NO2_max(img):
#     max = img.reduceRegion(reducer=ee.Reducer.max(), geometry=poi, scale=1120).get('tropospheric_NO2_column_number_density')
#     return img.set('date', img.date().format()).set('max',max)

# def UVAI_max(img):
#     max = img.reduceRegion(reducer=ee.Reducer.max(), geometry=poi, scale=1120).get('absorbing_aerosol_index')
#     return img.set('date', img.date().format()).set('max',max)

# def CH4_max(img):
#     max = img.reduceRegion(reducer=ee.Reducer.max(), geometry=poi, scale=1120).get('CH4_column_volume_mixing_ratio_dry_air_bias_corrected')
#     return img.set('date', img.date().format()).set('max',max)


# # Apply function to all images in collection
# CO_reduced_imgs = collCO.map(CO_max)

# NO2_reduced_imgs = collNO2.map(NO2_max)

# UVAI_reduced_imgs = collUVAI.map(UVAI_max)

# CH4_reduced_imgs = collCH4.map(CH4_max)


# def createFrame(imgs, inputList):
#     # Convert image information to list (date + mean value), create pandas df and set date column to index
#     img_list = imgs.reduceColumns(ee.Reducer.toList(2), inputList).values().get(0)
#     df = pd.DataFrame(img_list.getInfo(), columns=inputList)
#     df[inputList[0]] = pd.to_datetime(df[inputList[0]])
# #     df = df.set_index('date')
#     return df


# # Apply data frame function to collections
# dfCO = createFrame(CO_reduced_imgs, ['date','max'])
# dfNO2 = createFrame(NO2_reduced_imgs, ['date','max'])
# dfUVAI = createFrame(UVAI_reduced_imgs, ['date','max'])
# dfCH4 = createFrame(CH4_reduced_imgs, ['date','max'])


# # Create time series graph for POI

# # 1) create pyplot figure
# fig, ax = plt.subplots(figsize=(15,7))
# # ax2 = ax.twinx()

# # 2) add df to figure
# # ax.scatter(dfCO['date'], dfCO['max'], label='mean', marker='o')
# # ax.scatter(dfNO2['date'], dfNO2['max'], label='mean', marker='o')
# # ax.scatter(dfUVAI['date'], dfUVAI['max'], label='mean', marker='o')
# ax.scatter(dfCH4['date'], dfCH4['max'], label='max', marker='o')
# # ax2.scatter(dfNO2['date'], dfNO2['RH'], label='mean', marker='d', color='C1')

# # 3) set figure viz
# # ax.set_ylabel('CO_column_number_density mol/m^2', fontsize=20)
# # ax.set_ylabel('trop_NO2_column_number_dens mol/m^2', fontsize=20)
# # ax.set_ylabel('absorbing_aerosol_index (unitless)', fontsize=20)
# ax.set_ylabel('CH4_column_mix_ratio_bias_corr ppbV', fontsize=20)
# ax.set_ylim(ymin=1820, ymax=1910)
# ax.set_xlabel('Date', fontsize=20)
# ax.set_title('CH4', fontsize=20)
# ax.legend(loc='lower right')
# ax.grid()

In [12]:
## Retrieve information on L2 processor version for ImageCollection
# collCO.aggregate_array('PROCESSOR_VERSION').getInfo()
# collNO2.aggregate_array('PROCESSOR_VERSION').getInfo()
# collUVAI.aggregate_array('PROCESSOR_VERSION').getInfo()
# collCH4.aggregate_array('PROCESSOR_VERSION').getInfo()

## Retrieve information on GEE system id (single image property) for images in ImageCollection
# collCO.aggregate_array('system:id').getInfo()
# collNO2.aggregate_array('system:id').getInfo()
# collUVAI.aggregate_array('system:id').getInfo()
# collCH4.aggregate_array('system:id').getInfo()

['COPERNICUS/S5P/OFFL/L3_CH4/20210612T010540_20210613T180108',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T024710_20210613T193022',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T042840_20210613T213102',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T061010_20210613T231054',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T075140_20210614T011205',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T093310_20210614T023818',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T111440_20210614T045307',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T125610_20210614T061828',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T143739_20210614T074951',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T161909_20210614T091022',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T180039_20210614T110857',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T194209_20210614T124914',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T212339_20210614T141703',
 'COPERNICUS/S5P/OFFL/L3_CH4/20210612T230509_20210614T160421']

In [13]:
# Create single TROPOMI images by system ID
imgCO = ee.Image('COPERNICUS/S5P/OFFL/L3_CO/20210612T093310_20210613T231545').select('CO_column_number_density')
imgNO2 = ee.Image('COPERNICUS/S5P/OFFL/L3_NO2/20210612T093310_20210614T023822').select('tropospheric_NO2_column_number_density')
imgUVAI = ee.Image('COPERNICUS/S5P/OFFL/L3_AER_AI/20210612T093310_20210613T231547').select('absorbing_aerosol_index')
imgCH4 = ee.Image('COPERNICUS/S5P/OFFL/L3_CH4/20210612T093310_20210614T023818').select('CH4_column_volume_mixing_ratio_dry_air_bias_corrected')

# Double-check image properties
# imgCO.getInfo()

In [14]:
# Add layers to map
# Map.addLayer(S2_img, vizParamsS2, "S2")
# Map.addLayer(imgCO, vizParamsCO, "CO")
# Map.addLayer(imgNO2, vizParamsNO2, "NO2")
# Map.addLayer(imgUVAI, vizParamsUVAI, "UVAI")
# Map.addLayer(imgCH4, vizParamsCH4, "CH4")
# Map.addLayer(poi, {}, 'POI')
# Map.addLayer(GF, {}, 'GF')


# # Add colour scalebar to map
# colors = vizParamsCH4['palette']
# vmin = vizParamsCH4['min']
# vmax = vizParamsCH4['max']

# # Choose which product's colour scalebar to add
# Map.add_colorbar(vizParamsCO, label='CO column density (mol/m^2)')
# Map.add_colorbar(vizParamsNO2, label='NO2 tropospheric column density (mol/m^2)')
# Map.add_colorbar(vizParamsUVAI, label='Absorbing Aerosol Index')
# Map.add_colorbar(vizParamsCH4, label='CH4 column dry air mixing ratio (ppb)')


# # Visualise map
# Map.addLayerControl()
# Map.centerObject(poi, 10);
# Map

Map(center=[31.27227205178074, 48.116927066570454], controls=(WidgetControl(options=['position', 'transparent_…